# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
%%capture
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

# Question 1

In [8]:
#Import necessary modules
import pandas as pd

In [9]:
#Load json file as a table and understand basic attributes
df = pd.read_json('data/world_bank_projects.json')
#print(df.head())
#print(df.shape)
#print(df.columns)

In [10]:
# Using value_count on country name to determine top 10 countries with most project, convert it into a dataframe
most_prj = pd.DataFrame(df.countryname.value_counts().head(10)).reset_index()
most_prj.columns = ['Country','Project Count']
most_prj

,Country,Project Count
0,Republic of Indonesia,19
1,People's Republic of China,19
2,Socialist Republic of Vietnam,17
3,Republic of India,16
4,Republic of Yemen,13
5,Nepal,12
6,People's Republic of Bangladesh,12
7,Kingdom of Morocco,12
8,Republic of Mozambique,11
9,Africa,11


# Question 2

In [11]:
# Import necessary modules
import json
from pandas.io.json import json_normalize

In [12]:
# Check the mjtheme_namecode column. We see that the values are nested list
df.mjtheme_namecode.head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

In [13]:
# load json as string
json_str = json.load((open('data/world_bank_projects.json')))

In [14]:
# Normalize the data on the mjtheme_namecode column
theme_df = json_normalize(json_str, 'mjtheme_namecode')
theme_df.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [15]:
# Find out the codes of the 10 most popular themes, exclude the missing names and convert it into a data frame
most_theme = pd.DataFrame(theme_df[theme_df.name != ''].name.value_counts().head(10)).reset_index()
most_theme.columns = ["Theme name", "Theme count"]
most_theme

,Theme name,Theme count
0,Environment and natural resources management,223
1,Rural development,202
2,Human development,197
3,Public sector governance,184
4,Social protection and risk management,158
5,Financial and private sector development,130
6,Social dev/gender/inclusion,119
7,Trade and integration,72
8,Urban development,47
9,Economic management,33


# Question 3

In [16]:
# Import necessary modules.
import numpy as np

In [17]:
# Assess the missingness of the data.
n_missing = len(theme_df[theme_df.name == ''])
print('there are ' + str(n_missing) + ' missing values')

there are 122 missing values


In [18]:
theme_df.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [19]:
# Create a dictionary with key-value pairs pertaining theme code: theme name
temp = theme_df[theme_df.name != ''].sort_values('code').drop_duplicates()
theme_dict = dict(zip(temp.code, temp.name))
theme_dict

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [20]:
# Map the code column to the value associated with the its code according to theme_dict
theme_df.name = theme_df.code.map(theme_dict)

In [21]:
# Assess the missingness of the data.
n_missing = len(theme_df[theme_df.name == ''])
print('there are ' + str(n_missing) + ' missing values')

there are 0 missing values


# Alternative Forward Filling medthod

# To work with these values using Numpy, we have to convert all the blanks into NaN.
theme_df = theme_df.replace(r'^\s*$', np.nan, regex=True)

# Our tactics is Filling in missing values using forward fills. We first sort the theme_df dataframe by the code.
theme_fill = theme_df.sort_values(['code', 'name'])
theme_fill.tail(5)

# Then we forward fill the blank values.
theme_fill = theme_fill.ffill(axis = 'rows')
theme_fill.tail(5)

# We check to see if all the missing values have been filled
n_missing = len(theme_fill[theme_fill.name == ''])
print('there are ' + str(n_missing) + ' missing values')

# Now the missing values of our theme_fill dataframe has been filled. We re-sort the data frame in the correct order
theme_df_fill = theme_fill.sort_index()
theme_df_fill.head(10)